In [88]:
!pip3 install alpaca_trade_api schedule holidays

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [166]:
import os, sys
import alpaca_trade_api as api
import numpy as np
import pandas as pd
import glob

from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent,DRLEnsembleAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline

from stable_baselines3 import DDPG
from stable_baselines3.common.noise import (
    NormalActionNoise,
    OrnsteinUhlenbeckActionNoise,)
from stable_baselines3 import A2C
from stable_baselines3 import PPO
from stable_baselines3 import TD3
from stable_baselines3.td3.policies import MlpPolicy
from stable_baselines3.common.noise import (
    NormalActionNoise,
    OrnsteinUhlenbeckActionNoise,
)

from stable_baselines3 import SAC

import itertools
from datetime import datetime, time
import schedule
import holidays

holiday_check = holidays.US()

In [167]:
from importlib import reload # python 2.7 does not require this
import finrl.model.models
reload(finrl.model.models)
from finrl.model.models import DRLEnsembleAgent

import finrl.env.env_stocktrading 
reload(finrl.env.env_stocktrading)
from finrl.env.env_stocktrading import StockTradingEnv

import finrl.preprocessing.data
reload(finrl.preprocessing.data)
from finrl.preprocessing.data import data_split


In [169]:
api = api.REST('PK16U0CS4BSI2T7LLNJ7',
        'Yh0OAxgUzUUhhiSzxQBgvpEUa7nc8lVhhzFVKK0B',
        'https://paper-api.alpaca.markets', api_version='v2')


In [104]:
os.getcwd()

'/Users/quanphan/Documents/Project Development/finrl_adopt_application'

In [105]:
def load_model(agent, model_name, env):
    
    MODELS = {"a2c": A2C, "ddpg": DDPG, "td3": TD3, "sac": SAC, "ppo": PPO}
    
#     model = get_latest_file(config.TRAINED_MODEL_DIR, model_name)
    model = get_latest_file('./old_trained_models/', model_name)
    model = MODELS[agent].load(model, env=env)
    
    return model

In [93]:
def train_model_daily(model, model_name, tb_log_name, total_timesteps=5000):
    
    model = model.learn(total_timesteps=total_timesteps, tb_log_name=tb_log_name)
    model.save(f"{config.TRAINED_MODEL_DIR}/{model_name.upper()}_{total_timesteps//1000}k")

    return model

In [94]:
def perform_trade(actions_trade):
    
    for ticker in actions_trade.columns[1:]:
        if actions_trade.iloc[-1][ticker] > 0:
            api.submit_order(
                symbol=ticker,
                qty=int(actions_trade.iloc[-1][ticker]),
                side='buy',
                type='market',
                time_in_force='gtc'
            )
        elif actions_trade.iloc[-1][ticker] < 0:
            api.submit_order(
                symbol=ticker,
                qty= abs(int(actions_trade.iloc[-1][ticker])),
                side='sell',
                type='market',
                time_in_force='gtc'
            )

In [126]:
def get_latest_file(directory, file_name):
    files = glob.glob(os.path.join(os.getcwd(), directory, file_name))
#     files = glob.glob(os.path.join('/Users/evienguyen/Documents/FinRL/Modified FinRL', 'results', 'actions_trade_ensemble*'))

    files.sort(key=os.path.getmtime)
    print(files)
    actions_trade =  sorted(files,key=os.path.getmtime)[-1]
    
    return actions_trade

In [113]:
from datetime import datetime, timedelta

validation_window = 63
rebalance_window = 63

today = datetime.today().strftime('%Y-%m-%d')
beginning = '2000-01-01'
train_start = datetime.today() - timedelta(days=validation_window + rebalance_window)

val_start = datetime.today() - timedelta(days=validation_window)
val_start = val_start.strftime('%Y-%m-%d')

val_end = datetime.today()
val_end = val_end.strftime('%Y-%m-%d')

raw_full_path = os.path.join(os.getcwd(), 'data', 'raw_full.csv')

if not os.path.exists(raw_full_path):
    print("Not found existing data")
    
    raw_full = YahooDownloader(start_date = beginning,
                         end_date = '2021-06-23',
                         ticker_list = config.SP_500_TICKER).fetch_data()

    raw_full.to_csv(raw_full_path)
else:
    print("Found existing data")
    raw_full = pd.read_csv(raw_full_path)
    
    latest_date = raw_full.iloc[-1, :].date
    print(today, latest_date)
    if today != latest_date:
        raw_full.append(
            YahooDownloader(start_date = latest_date,
                             end_date = today,
                             ticker_list = config.SP_500_TICKER).fetch_data()
        )

init_thresh = calc_turb_thresh(raw_full)

Found existing data
2021-07-06 2021-06-22
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 co

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CTL: No data found, symbol may be delisted
[*********************100%************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- UTX: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- VAR: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- VIAB: No data found,

In [154]:
def refresh_daily_data(train_start, today):
    '''
        Return full processed 126 days data with turbulence + tech_indicators
    '''
    
    raw_recent_path = os.path.join('data', 'raw_recent')
    
    if not os.path.exists(raw_recent_path):
        
        raw_recent = YahooDownloader(start_date = train_start - timedelta(days=365),
                         end_date = today,
                         ticker_list = config.SP_500_TICKER).fetch_data()
    else:
        raw_recent = pd.read_csv(raw_recent_path)
        
        latest_date = raw_recent.iloc[-1, :].date
        
        if today != latest_date:
            raw_recent.append(
                YahooDownloader(start_date = latest_date,
                                 end_date = today,
                                 ticker_list = config.SP_500_TICKER).fetch_data()
            )
        else:
            raw_recent.iloc[-1, :] = YahooDownloader(start_date = today,
                                 end_date = today,
                                 ticker_list = config.SP_500_TICKER).fetch_data()
    
    fe = FeatureEngineer(
                        use_technical_indicator=True,
                        tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                        use_turbulence=True,
                        user_defined_feature = False)

    processed = fe.preprocess_data(raw_recent)
    
    list_ticker = processed["tic"].unique().tolist()
    list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
    combination = list(itertools.product(list_date,list_ticker))

    processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
    processed_full = processed_full[processed_full['date'].isin(processed['date'])]
    processed_full = processed_full.sort_values(['date','tic'])

    processed_full = processed_full.fillna(0)
    
    return processed_full, raw_recent

In [97]:
def calc_turb_thresh(data):
    '''
        Input: raw full data df
        Return initial turbulence from full data until trading day
    '''
    
    df_price_pivot = data.pivot(index="date", columns="tic", values="close")
    # use returns to calculate turbulence
    df_price_pivot = df_price_pivot.pct_change()
    
    print(df_price_pivot.shape)
    print(df_price_pivot)
    
    unique_date = data.date.unique()
    # start after a year
    start = 252
    turbulence_index = [0] * start
    # turbulence_index = [0]
    count = 0
    for i in range(start, len(unique_date)):
        current_price = df_price_pivot[df_price_pivot.index == unique_date[i]]
        # use one year rolling window to calcualte covariance
        hist_price = df_price_pivot[
            (df_price_pivot.index < unique_date[i])
            & (df_price_pivot.index >= unique_date[i - 252])
        ]
        # Drop tickers which has number missing values more than the "oldest" ticker
        filtered_hist_price = hist_price.iloc[hist_price.isna().sum().min():].dropna(axis=1)

        cov_temp = filtered_hist_price.cov()
        current_temp = current_price[[x for x in filtered_hist_price]] - np.mean(filtered_hist_price, axis=0)
        temp = current_temp.values.dot(np.linalg.pinv(cov_temp)).dot(
            current_temp.values.T
        )
        if temp > 0:
            count += 1
            if count > 2:
                turbulence_temp = temp[0][0]
            else:
                # avoid large outlier because of the calculation just begins
                turbulence_temp = 0
        else:
            turbulence_temp = 0
        turbulence_index.append(turbulence_temp)

    turbulence_index = pd.DataFrame(
        {"date": df_price_pivot.index, "turbulence": turbulence_index}
    )
    
    data = data.merge(turbulence_index, on="date")
    data = data.sort_values(["date", "tic"]).reset_index(drop=True)
    
    insample_turbulence_threshold = np.quantile(data.turbulence.values, 0.85)
    
    return insample_turbulence_threshold

In [163]:
def daily_predict(df, raw_recent, initial_turbulence):
    
    timesteps_dict = {'a2c' : 30_000, 
                 'ppo' : 100_000, 
                 'ddpg' : 10_000
                 }
    
    last_state_ensemble = []
    insample_turbulence_threshold = initial_turbulence

    ppo_sharpe_list = []
    ddpg_sharpe_list = []
    a2c_sharpe_list = []

    model_use = ''
    # start = time.time()
    stock_dimension = len(df.tic.unique())
    state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
    print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

    env_kwargs = {
        "hmax": 100, 
        "initial_amount": 100000, #TODO: dynamic change amount
        "buy_cost_pct": 0.001, 
        "sell_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity":5

    }
    ensemble_agent = DRLEnsembleAgent(df=df,
                     rebalance_window=rebalance_window, 
                     validation_window=validation_window, 
                     **env_kwargs)

#     validation_start_date_list.append(val_start)
#     validation_end_date_list.append(val_end)
    # iteration_list.append(i)

    print("============================================")
    ## initial state is empty
    # if i - self.rebalance_window - self.validation_window == 0:
    #     # inital state
    #     initial = True
    # else:
    #     # previous state
    #     initial = False

    # Tuning trubulence index based on historical data
    # Turbulence lookback window is one quarter (63 days)
#     end_date_index = df.index[df["date"] == ensemble_agent.unique_trade_date[train_start]].to_list()[-1]
#     start_date_index = end_date_index - 63 + 1

#     historical_turbulence = df.iloc[start_date_index:(end_date_index + 1), :]

#     historical_turbulence = historical_turbulence.drop_duplicates(subset=['date'])

    historical_turbulence_mean = np.mean(df.iloc[64:, :].turbulence.values)

    
       
    turbulence_index = calc_turb_thresh(raw_recent)
    print(historical_turbulence_mean)
    print(turbulence_index)
    
    if historical_turbulence_mean > insample_turbulence_threshold:
        # if the mean of the historical data is greater than the 90% quantile of insample turbulence data
        # then we assume that the current market is volatile,
        # therefore we set the 90% quantile of insample turbulence data as the turbulence threshold
        # meaning the current turbulence can't exceed the 90% quantile of insample turbulence data
        turbulence_threshold = insample_turbulence_threshold
    else:
        # if the mean of the historical data is less than the 90% quantile of insample turbulence data
        # then we tune up the turbulence_threshold, meaning we lower the risk
        turbulence_threshold = np.quantile(turbulence_index, 1)
    print("turbulence_threshold: ", turbulence_threshold)

    ############## Environment Setup starts ##############
    ## training env
#     train = data_split(ensemble_agent.df, start=train_start, end=val_start)
#     ensemble_agent.train_env = DummyVecEnv([lambda: StockTradingEnv(train,
#                                                         ensemble_agent.stock_dim,
#                                                         ensemble_agent.hmax,
#                                                         ensemble_agent.initial_amount,
#                                                         ensemble_agent.buy_cost_pct,
#                                                         ensemble_agent.sell_cost_pct,
#                                                         ensemble_agent.reward_scaling,
#                                                         ensemble_agent.state_space,
#                                                         ensemble_agent.action_space,
#                                                         ensemble_agent.tech_indicator_list,
#                                                         print_verbosity=ensemble_agent.print_verbosity)])

    validation = data_split(ensemble_agent.df, start = df.date.unique()[-63], end = df.date.unique()[-1])
    ############## Environment Setup ends ##############

    ############## Validation starts ##############

    print("======A2C Validation from: ", val_start, "to ",val_end)
    val_env_a2c = DummyVecEnv([lambda: StockTradingEnv(validation,
                                                        ensemble_agent.stock_dim,
                                                        ensemble_agent.hmax,
                                                        ensemble_agent.initial_amount,
                                                        ensemble_agent.buy_cost_pct,
                                                        ensemble_agent.sell_cost_pct,
                                                        ensemble_agent.reward_scaling,
                                                        ensemble_agent.state_space,
                                                        ensemble_agent.action_space,
                                                        ensemble_agent.tech_indicator_list,
                                                        turbulence_threshold=turbulence_threshold,
#                                                         iteration=i,
                                                        model_name='A2C',
                                                        mode='validation',
                                                        print_verbosity=ensemble_agent.print_verbosity)])
    val_obs_a2c = val_env_a2c.reset()

    model_a2c = load_model('a2c', 'A2C_30k*', env=val_env_a2c)
    
    val_a2c_model =  ensemble_agent.DRL_validation(model=model_a2c,test_data=validation,test_env=val_env_a2c,test_obs=val_obs_a2c)
    sharpe_a2c = ensemble_agent.get_validation_sharpe(model_name="A2C")
    print("A2C Sharpe Ratio: ", sharpe_a2c)

    print("======PPO Validation from: ", val_start, "to ",val_end)
    val_env_ppo = DummyVecEnv([lambda: StockTradingEnv(validation,
                                                        ensemble_agent.stock_dim,
                                                        ensemble_agent.hmax,
                                                        ensemble_agent.initial_amount,
                                                        ensemble_agent.buy_cost_pct,
                                                        ensemble_agent.sell_cost_pct,
                                                        ensemble_agent.reward_scaling,
                                                        ensemble_agent.state_space,
                                                        ensemble_agent.action_space,
                                                        ensemble_agent.tech_indicator_list,
                                                        turbulence_threshold=turbulence_threshold,
#                                                         iteration=i,
                                                        model_name='PPO',
                                                        mode='validation',
                                                        print_verbosity=ensemble_agent.print_verbosity)])
    val_obs_ppo = val_env_ppo.reset()
    
    model_ppo = load_model('ppo', 'PPO_100k*', env=val_env_ppo)
    
    val_ppo_model = ensemble_agent.DRL_validation(model=model_ppo,test_data=validation,test_env=val_env_ppo,test_obs=val_obs_ppo)
    sharpe_ppo = ensemble_agent.get_validation_sharpe(model_name="PPO")
    print("PPO Sharpe Ratio: ", sharpe_ppo)

    print("======DDPG Validation from: ", val_start, "to ",val_end)
    val_env_ddpg = DummyVecEnv([lambda: StockTradingEnv(validation,
                                                        ensemble_agent.stock_dim,
                                                        ensemble_agent.hmax,
                                                        ensemble_agent.initial_amount,
                                                        ensemble_agent.buy_cost_pct,
                                                        ensemble_agent.sell_cost_pct,
                                                        ensemble_agent.reward_scaling,
                                                        ensemble_agent.state_space,
                                                        ensemble_agent.action_space,
                                                        ensemble_agent.tech_indicator_list,
                                                        turbulence_threshold=turbulence_threshold,
#                                                         iteration=i,
                                                        model_name='DDPG',
                                                        mode='validation',
                                                        print_verbosity=ensemble_agent.print_verbosity)])
    val_obs_ddpg = val_env_ddpg.reset()
    
    model_ddpg = load_model('ddpg', 'DDPG_10k*', env=val_env_ddpg)
    
    val_ddpg_model = ensemble_agent.DRL_validation(model=model_ddpg,test_data=validation,test_env=val_env_ddpg,test_obs=val_obs_ddpg)
    sharpe_ddpg = ensemble_agent.get_validation_sharpe(model_name="DDPG")
    print("DDPG Sharpe Ratio: ", sharpe_ddpg)

    ppo_sharpe_list.append(sharpe_ppo)
    a2c_sharpe_list.append(sharpe_a2c)
    ddpg_sharpe_list.append(sharpe_ddpg)

    print("======Best Model Retraining from: ", val_start, "to ",
          val_end)
    # Environment setup for model retraining up to first trade date
    train_full = data_split(ensemble_agent.df, start = df.date.unique()[-63], end = df.date.unique()[-1])
    train_full_env = DummyVecEnv([lambda: StockTradingEnv(train_full,
                                                        ensemble_agent.stock_dim,
                                                        ensemble_agent.hmax,
                                                        ensemble_agent.initial_amount,
                                                        ensemble_agent.buy_cost_pct,
                                                        ensemble_agent.sell_cost_pct,
                                                        ensemble_agent.reward_scaling,
                                                        ensemble_agent.state_space,
                                                        ensemble_agent.action_space,
                                                        ensemble_agent.tech_indicator_list,
                                                        print_verbosity=ensemble_agent.print_verbosity)])
    # Model Selection based on sharpe ratio
    if (sharpe_ppo >= sharpe_a2c) & (sharpe_ppo >= sharpe_ddpg):
        model_use = 'ppo'
        model_ensemble = load_model('ppo', 'PPO_100k*', env=train_full_env)

    elif (sharpe_a2c > sharpe_ppo) & (sharpe_a2c > sharpe_ddpg):
        model_use = 'a2c'
        model_ensemble = load_model('a2c', 'A2C_30k*', env=train_full_env)
    else:
        model_use = 'ddpg'
        model_ensemble = load_model('ddpg', 'DDPG_10k*', env=train_full_env)
        
    model_ensemble = train_model_daily(model_ensemble, "ensemble", tb_log_name="ensemble_{}".format(model_use), total_timesteps=timesteps_dict[model_use]) #50_000
    ############## Training and Validation ends ##############

    ############## Trading starts ##############
    # TODO: Update last state from correct environment
    print("======Trading from: ", val_start, "to ", val_end)
    #print("Used Model: ", model_ensemble)
    last_state_ensemble = ensemble_agent.DRL_prediction(model=model_ensemble, name="ensemble",
                                             last_state=last_state_ensemble,
                                             turbulence_threshold = turbulence_threshold,
                                             initial=(len(last_state_ensemble) == 0),
                                             trade_data=train_full)
    ############## Trading ends #############

In [101]:
def main():
    in_trading_time = (
        time(hour=9,minute=30) <= datetime.now().time() < time(hour=16, minute=0) and
        (datetime.now().date() not in holiday_check)
    )
    print("DATE: {} HOUR: {} \nHOLIDAY: {}".format(datetime.now().date(), datetime.now().time(), (datetime.now().date() in holiday_check)))
    
    if in_trading_time:
        
        daily_processed_df, raw_recent = refresh_daily_data(train_start, today)
        daily_predict(daily_processed_df, raw_recent, init_thresh)

        actions_trade = pd.read_csv(get_latest_file('results','actions_trade_ensemble*'))
        perform_trade(actions_trade=actions_trade)
        
    else:
        print("Trading close")
      
schedule.clear()
schedule.every(15).minutes.do(main)

Every 15 minutes do main() (last run: [never], next run: 2021-07-06 12:48:44)

In [102]:
schedule.get_jobs()

[Every 15 minutes do main() (last run: [never], next run: 2021-07-06 12:48:44)]

In [103]:
while True:
    schedule.run_pending()

KeyboardInterrupt: 

In [168]:
# daily_processed_df, raw_recent = refresh_daily_data(train_start, today)
daily_predict(daily_processed_df, raw_recent, init_thresh)

actions_trade = pd.read_csv(get_latest_file('results','actions_trade_ensemble*'))
perform_trade(actions_trade=actions_trade)

Stock Dimension: 478, State Space: 4781
(339, 478)
tic                A       AAL       AAP      AAPL      ABBV       ABC  \
date                                                                     
2020-03-02       NaN       NaN       NaN       NaN       NaN       NaN   
2020-03-03 -0.020129 -0.053553 -0.020001 -0.031759 -0.012628 -0.014800   
2020-03-04  0.050174  0.038095  0.019719  0.046385  0.047733  0.052521   
2020-03-05 -0.046117 -0.134377 -0.044996 -0.032437 -0.012425 -0.037730   
2020-03-06 -0.023614 -0.004364  0.007800 -0.013280 -0.019755 -0.013338   
...              ...       ...       ...       ...       ...       ...   
2021-06-28  0.004413 -0.037354  0.000391  0.012546  0.000177 -0.013753   
2021-06-29  0.005949 -0.014493  0.003863  0.011500 -0.006195 -0.002615   
2021-06-30 -0.006720  0.006167 -0.000779  0.004621  0.003028  0.000437   
2021-07-01 -0.000135  0.014144  0.018524  0.002263  0.014382  0.017032   
2021-07-02  0.008293 -0.001395  0.010099  0.019596  0.007964 

/Users/quanphan/Documents/Project Development/finrl_adopt_application/finrl/env/env_stocktrading.py:206: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "index" which is no longer supported as of 3.3 and will become an error two minor releases later
  plt.savefig('results/account_value_{}_{}_daily.png'.format(self.mode,self.model_name,),index=False)


A2C Sharpe Ratio:  0.3635651924839796
======PPO Validation from:  2021-05-04 to  2021-07-06
['/Users/quanphan/Documents/Project Development/finrl_adopt_application/./old_trained_models/PPO_100k_126.zip']
PPO Sharpe Ratio:  -0.06427353013627451
======DDPG Validation from:  2021-05-04 to  2021-07-06
['/Users/quanphan/Documents/Project Development/finrl_adopt_application/./old_trained_models/DDPG_10k_126.zip']
DDPG Sharpe Ratio:  -0.2348438265349017
======Best Model Retraining from:  2021-05-04 to  2021-07-06
['/Users/quanphan/Documents/Project Development/finrl_adopt_application/./old_trained_models/A2C_30k_126.zip']
Logging to tensorboard_log/a2c/ensemble_a2c_3
day: 61, episode: 5
begin_total_asset: 100000.00
end_total_asset: 103690.01
total_reward: 3690.01
total_cost: 157.44
total_trades: 14749
Sharpe: 0.728
------------------------------------
| environment/          |          |
|    portfolio_value    | 1e+05    |
|    total_cost         | 126      |
|    total_reward       | 290   

------------------------------------
| environment/          |          |
|    portfolio_value    | 1.01e+05 |
|    total_cost         | 201      |
|    total_reward       | 923      |
|    total_reward_pct   | 0.923    |
|    total_trades       | 14626    |
| time/                 |          |
|    fps                | 95       |
|    iterations         | 800      |
|    time_elapsed       | 41       |
|    total_timesteps    | 4000     |
| train/                |          |
|    entropy_loss       | -744     |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 6799     |
|    policy_loss        | -72.8    |
|    std                | 1.15     |
|    value_loss         | 0.021    |
------------------------------------
day: 61, episode: 65
begin_total_asset: 100000.00
end_total_asset: 102368.72
total_reward: 2368.72
total_cost: 234.11
total_trades: 14541
Sharpe: 0.517
day: 61, episode: 70
begin_total_asset: 100000.00
end_total_asset: 1068

day: 61, episode: 125
begin_total_asset: 100000.00
end_total_asset: 109434.20
total_reward: 9434.20
total_cost: 214.82
total_trades: 14777
Sharpe: 2.198
------------------------------------
| environment/          |          |
|    portfolio_value    | 1.06e+05 |
|    total_cost         | 253      |
|    total_reward       | 6.33e+03 |
|    total_reward_pct   | 6.33     |
|    total_trades       | 14658    |
| time/                 |          |
|    fps                | 87       |
|    iterations         | 1600     |
|    time_elapsed       | 91       |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -771     |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 7599     |
|    policy_loss        | 2.82     |
|    std                | 1.21     |
|    value_loss         | 0.00401  |
------------------------------------
day: 61, episode: 130
begin_total_asset: 100000.00
end_total_asset: 10

...

day: 61, episode: 255
begin_total_asset: 100000.00
end_total_asset: 103055.34
total_reward: 3055.34
total_cost: 181.39
total_trades: 14262
Sharpe: 0.632
------------------------------------
| environment/          |          |
|    portfolio_value    | 1.03e+05 |
|    total_cost         | 192      |
|    total_reward       | 2.71e+03 |
|    total_reward_pct   | 2.71     |
|    total_trades       | 14258    |
| time/                 |          |
|    fps                | 82       |
|    iterations         | 3200     |
|    time_elapsed       | 192      |
|    total_timesteps    | 16000    |
| train/                |          |
|    entropy_loss       | -824     |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 9199     |
|    policy_loss        | 90.3     |
|    std                | 1.36     |
|    value_loss         | 0.0139   |
------------------------------------
day: 61, episode: 260
begin_total_asset: 100000.00
end_total_asset: 10

day: 61, episode: 315
begin_total_asset: 100000.00
end_total_asset: 98058.59
total_reward: -1941.41
total_cost: 286.62
total_trades: 14451
Sharpe: -0.180
day: 61, episode: 320
begin_total_asset: 100000.00
end_total_asset: 103031.73
total_reward: 3031.73
total_cost: 277.39
total_trades: 14417
Sharpe: 0.586
------------------------------------
| environment/          |          |
|    portfolio_value    | 9.95e+04 |
|    total_cost         | 124      |
|    total_reward       | -500     |
|    total_reward_pct   | -0.5     |
|    total_trades       | 14348    |
| time/                 |          |
|    fps                | 82       |
|    iterations         | 4000     |
|    time_elapsed       | 242      |
|    total_timesteps    | 20000    |
| train/                |          |
|    entropy_loss       | -852     |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 9999     |
|    policy_loss        | 34.7     |
|    std                | 1

day: 61, episode: 380
begin_total_asset: 100000.00
end_total_asset: 105203.95
total_reward: 5203.95
total_cost: 173.13
total_trades: 14219
Sharpe: 1.215
day: 61, episode: 385
begin_total_asset: 100000.00
end_total_asset: 109117.15
total_reward: 9117.15
total_cost: 206.84
total_trades: 14122
Sharpe: 2.191
------------------------------------
| environment/          |          |
|    portfolio_value    | 1.04e+05 |
|    total_cost         | 229      |
|    total_reward       | 4.22e+03 |
|    total_reward_pct   | 4.22     |
|    total_trades       | 14111    |
| time/                 |          |
|    fps                | 84       |
|    iterations         | 4800     |
|    time_elapsed       | 284      |
|    total_timesteps    | 24000    |
| train/                |          |
|    entropy_loss       | -879     |
|    explained_variance | -0.672   |
|    learning_rate      | 0.0005   |
|    n_updates          | 10799    |
|    policy_loss        | -583     |
|    std                | 1.

day: 61, episode: 445
begin_total_asset: 100000.00
end_total_asset: 110061.71
total_reward: 10061.71
total_cost: 208.20
total_trades: 14254
Sharpe: 2.169
day: 61, episode: 450
begin_total_asset: 100000.00
end_total_asset: 107791.97
total_reward: 7791.97
total_cost: 202.87
total_trades: 14288
Sharpe: 1.958
------------------------------------
| environment/          |          |
|    portfolio_value    | 1.08e+05 |
|    total_cost         | 224      |
|    total_reward       | 8.44e+03 |
|    total_reward_pct   | 8.44     |
|    total_trades       | 14386    |
| time/                 |          |
|    fps                | 85       |
|    iterations         | 5600     |
|    time_elapsed       | 328      |
|    total_timesteps    | 28000    |
| train/                |          |
|    entropy_loss       | -904     |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 11599    |
|    policy_loss        | 76.2     |
|    std                | 1

AttributeError: module 'alpaca_trade_api' has no attribute 'submit_order'

In [170]:
actions_trade = pd.read_csv(get_latest_file('results','actions_trade_ensemble*'))
perform_trade(actions_trade=actions_trade)

['/Users/quanphan/Documents/Project Development/finrl_adopt_application/results/actions_trade_ensemble_daily.csv']
